In [176]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utils.preprocess import *
from utils.model import *

# Read data

In [177]:
y = read_target_train()

# Preprocess data

In [178]:
X, X_test = read_preprocessed_data()

# Train catboost

In [179]:
from catboost import CatBoostRegressor

In [180]:
model_tst = CatBoostRegressor(iterations = 300, verbose = 50, depth = 3, random_state = config['random_state'], cat_features=get_cat_features_list(X), loss_function='MAE')
model_C = CatBoostRegressor(iterations = 300, verbose = 50, depth = 3, random_state = config['random_state'], cat_features=get_cat_features_list(X), loss_function='MAE')

In [181]:
model_tst.fit(X, y['TST'])

0:	learn: 23.6516422	total: 14.3ms	remaining: 4.26s
50:	learn: 18.6522370	total: 681ms	remaining: 3.32s
100:	learn: 17.2961875	total: 1.34s	remaining: 2.64s
150:	learn: 16.5263993	total: 2s	remaining: 1.97s
200:	learn: 15.8562281	total: 2.65s	remaining: 1.31s
250:	learn: 15.3130964	total: 3.31s	remaining: 647ms
299:	learn: 14.8388852	total: 3.97s	remaining: 0us


In [182]:
model_C.fit(X, y['C'])

0:	learn: 0.0386563	total: 14.5ms	remaining: 4.33s
50:	learn: 0.0311469	total: 699ms	remaining: 3.41s
100:	learn: 0.0274317	total: 1.36s	remaining: 2.69s
150:	learn: 0.0237275	total: 2.03s	remaining: 2s
200:	learn: 0.0216954	total: 2.7s	remaining: 1.33s
250:	learn: 0.0210527	total: 3.36s	remaining: 655ms
299:	learn: 0.0203235	total: 4s	remaining: 0us


In [115]:
save_catboost_models(model_tst, model_C)

In [183]:
args = model_C.feature_importances_.argsort()[::-1][:250]
# plt.barh(np.array(model_C.feature_names_)[args], model_C.feature_importances_[args]) 
C_args = np.array(model_C.feature_names_)[args]

In [184]:
args = model_tst.feature_importances_.argsort()[::-1][:250]
# plt.barh(np.array(model_tst.feature_names_)[args], model_tst.feature_importances_[args]) 
tst_args = np.array(model_tst.feature_names_)[args]

In [174]:
C_args

array(['plavka_NMZ_C', 'RAS_sum', 'plavka_NMZ_TST', 'NMSYP_изв_ЦОИ',
       'VES', 'AR_last', 'O2_sum', 'AR_end_Продувка', 'AR_amax',
       'CO2_last', 'plavka_NAPR_ZAD_TST', 'O2_end_Продувка',
       'CO2_end_Продувка', 'NMSYP_изв_ЦОИ_amax', 'CO_last'], dtype='<U32')

In [175]:
tst_args

array(['O2_sum', 'RAS_sum', 'plavka_NMZ_TST', 'O2_amin', 'O2_mean',
       'O2_first', 'CO2_start_Заливка чугуна', 'O2_idxmin', 'H2_mean',
       'O2_median', 'O2_last', 'T', 'CO2_median', 'O2_idxmax',
       'AR_end_Продувка'], dtype='<U32')

In [185]:
model_tst_3 = CatBoostRegressor(iterations = 400, verbose = 50, depth = 3, random_state = config['random_state'], cat_features=list(set(tst_args).intersection(get_cat_features_list(X))), loss_function='MAE')
model_C_3 = CatBoostRegressor(iterations = 400, verbose = 50, depth = 3, random_state = config['random_state'], cat_features=list(set(C_args).intersection(get_cat_features_list(X))), loss_function='MAE')

In [186]:
model_tst_3.fit(X[tst_args], y['TST'])

0:	learn: 23.6949481	total: 9.71ms	remaining: 3.87s
50:	learn: 18.4934200	total: 460ms	remaining: 3.15s
100:	learn: 17.1904224	total: 905ms	remaining: 2.68s
150:	learn: 16.3539364	total: 1.35s	remaining: 2.23s
200:	learn: 15.7915749	total: 1.8s	remaining: 1.78s
250:	learn: 15.2860249	total: 2.24s	remaining: 1.33s
300:	learn: 14.7781685	total: 2.71s	remaining: 892ms
350:	learn: 14.3064741	total: 3.16s	remaining: 441ms
399:	learn: 13.9358016	total: 3.6s	remaining: 0us


In [187]:
model_C_3.fit(X[C_args], y['C'])

0:	learn: 0.0387026	total: 9.66ms	remaining: 3.85s
50:	learn: 0.0308913	total: 474ms	remaining: 3.25s
100:	learn: 0.0267857	total: 930ms	remaining: 2.75s
150:	learn: 0.0239372	total: 1.39s	remaining: 2.29s
200:	learn: 0.0218012	total: 1.84s	remaining: 1.82s
250:	learn: 0.0209387	total: 2.3s	remaining: 1.36s
300:	learn: 0.0203927	total: 2.75s	remaining: 906ms
350:	learn: 0.0199086	total: 3.21s	remaining: 448ms
399:	learn: 0.0195841	total: 3.66s	remaining: 0us


In [188]:
save_catboost_models(model_tst_3, model_C_3)

### Model Validation

In [15]:
from catboost import Pool, cv

cv_data = X_tst_copy

labels = y['TST']

cat_features = get_cat_features_list(X)

cv_dataset = Pool(data=cv_data,
                  label=labels,
                  cat_features=cat_features)

params = {"iterations": 1200,
          "depth": 3,
          "loss_function": "MAE",
          "verbose": 50}

scores = cv(cv_dataset,
            params,
            fold_count=5,
            plot="True")

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 1599.5939094	test: 1599.5938682	best: 1599.5938682 (0)
50:	learn: 347.0403156	test: 347.1365096	best: 347.1365096 (50)
100:	learn: 74.4821040	test: 74.5827869	best: 74.5827869 (100)
150:	learn: 24.6276175	test: 25.1763558	best: 25.1763558 (150)
200:	learn: 17.9234498	test: 18.7811106	best: 18.7811106 (200)
250:	learn: 16.4739094	test: 17.6340312	best: 17.6340312 (250)
300:	learn: 15.5654434	test: 17.1245746	best: 17.1245746 (300)
350:	learn: 14.8655151	test: 16.7734657	best: 16.7734657 (350)
400:	learn: 14.3049505	test: 16.5672728	best: 16.5672728 (400)
450:	learn: 13.8289447	test: 16.3868281	best: 16.3868281 (450)
500:	learn: 13.4337461	test: 16.2706617	best: 16.2706617 (500)
550:	learn: 13.0775700	test: 16.1667441	best: 16.1667441 (550)
600:	learn: 12.7744884	test: 16.0859224	best: 16.0859224 (600)
650:	learn: 12.5049079	test: 16.0171485	best: 16.0171485 (650)
700:	learn: 12.2642187	test: 15.9708838	best: 15.9708838 (700)
750:	learn: 12.0471381	test: 15.9240092	best: 15.924

In [16]:
from catboost import Pool, cv

cv_data = X_C_copy

labels = y['C']

cat_features = get_cat_features_list(X)

cv_dataset = Pool(data=cv_data,
                  label=labels,
                  cat_features=cat_features)

params = {"iterations": 1200,
          "depth": 3,
#           "l2_leaf_reg" : 4,
          "loss_function": "MAE",
          "verbose": 50}

scores = cv(cv_dataset,
            params,
            fold_count=5, 
            plot="True")

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.0806666	test: 0.0806686	best: 0.0806686 (0)
50:	learn: 0.0389950	test: 0.0390236	best: 0.0390236 (50)
100:	learn: 0.0307673	test: 0.0311081	best: 0.0311081 (100)
150:	learn: 0.0269163	test: 0.0279508	best: 0.0279508 (150)
200:	learn: 0.0236489	test: 0.0251957	best: 0.0251957 (200)
250:	learn: 0.0215524	test: 0.0234192	best: 0.0234192 (250)
300:	learn: 0.0203976	test: 0.0226952	best: 0.0226952 (300)
350:	learn: 0.0198179	test: 0.0224798	best: 0.0224798 (350)
400:	learn: 0.0193994	test: 0.0223277	best: 0.0223277 (400)
450:	learn: 0.0191088	test: 0.0222438	best: 0.0222415 (448)
500:	learn: 0.0188521	test: 0.0221751	best: 0.0221751 (500)
550:	learn: 0.0186167	test: 0.0221277	best: 0.0221277 (550)
600:	learn: 0.0183732	test: 0.0220687	best: 0.0220687 (600)
650:	learn: 0.0181762	test: 0.0220394	best: 0.0220394 (650)
700:	learn: 0.0179898	test: 0.0220126	best: 0.0220119 (696)
750:	learn: 0.0178034	test: 0.0219813	best: 0.0219813 (750)
800:	learn: 0.0176286	test: 0.0219294	best: 0.